<a href="https://colab.research.google.com/github/hectorcamachoz/SVM-y-Multiple-Testing/blob/main/A3_1_594557.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM y Multiple Testing

En esta actividad se trabajara con la base de datos de la que se habló en mi clase de Inteligencia Artificial, que consiste de 83 muestras y 2308 variables de entrada, que consisten en la expresión génica estandarizada de distintos genes. La variable de salida cuenta con valores numéricos del 1 al 4 que
corresponden a distintos tipos de cáncer.

**1.** Lo primero que se realizara sera importar la base de datos, revisar que no tenga huecos. Tambien, se calculara la diferencia de promedios entre las clases 2 y 4, se imprimiran los 10 genes con mayor diferencia.


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('A3.1_Khan.csv')
print("Tabla con numero de huecos en variables: ")
print(df.isnull().sum())

df_2 = df[df['y'] == 2]
df_4 = df[df['y'] == 4]

gen_2 = df_2.drop(columns=['y'])
gen_4 = df_4.drop(columns=['y'])

promedios_2 = gen_2.mean()
promedios_4 = gen_4.mean()

dif = (promedios_2 - promedios_4).abs()

top_10_genes = dif.sort_values(ascending=False).head(10)

print("\nTop 10 genes con mayor diferencia de promedios entre clases 2 y 4:")
print(top_10_genes)



Tabla con numero de huecos en variables: 
X1       0
X2       0
X3       0
X4       0
X5       0
        ..
X2305    0
X2306    0
X2307    0
X2308    0
y        0
Length: 2309, dtype: int64

Top 10 genes con mayor diferencia de promedios entre clases 2 y 4:
X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
X246     1.837830
dtype: float64


Considero que la diferencia entre las clases puede indicar que podria haber una diferencia real entre la clase 2 y 4 y que no es un simple resultado de variacion al azar, y que podria haber una relacion entre la variable y la clase . En este caso el gen con mayor diferencia en ambas clases es la X187, y la decima variable con mayor diferencia fue X246

**2.** A continuacion calculare el estadistico t y el p value para comparar las medias de los genes entre la clase 2 y 4, Utilizando la metodologia de Bonferroni, Holm y Benajmini-Hochberg (para utilizar estas metodologias utilice chatGPT)

In [4]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
df_2_4 = df[df['y'].isin([2, 4])]
df_filtered = df_2_4.drop(columns=['y'])
t_stats = []
p_values = []

for gen in df_filtered.columns[1:]:  # Excluyendo columna 'clase'
    t, p = ttest_ind(df_2[gen], df_4[gen], equal_var=False)
    t_stats.append(t)
    p_values.append(p)

# Bonferroni
_, p_bonf, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

# Holm
_, p_holm, _, _ = multipletests(p_values, alpha=0.05, method='holm')

# Benjamini-Hochberg (FDR)
_, p_bh, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

resultados = pd.DataFrame({
    'Gen': df_filtered.columns[1:],
    't_stat': t_stats,
    'p_value': p_values,
    'p_bonf': p_bonf,
    'p_holm': p_holm,
    'p_bh': p_bh
})
significativos_bonf = resultados[resultados['p_bonf'] < 0.05]
significativos_holm = resultados[resultados['p_holm'] < 0.05]
significativos_bh = resultados[resultados['p_bh'] < 0.05]

# Imprime los genes significativos
print("Genes significativos con Bonferroni:\n", significativos_bonf)
print("\nGenes significativos con Holm:\n", significativos_holm)
print("\nGenes significativos con Benjamini-Hochberg:\n", significativos_bh)


Genes significativos con Bonferroni:
         Gen     t_stat       p_value        p_bonf        p_holm          p_bh
0        X2  -6.900138  7.383962e-08  1.703480e-04  1.681328e-04  5.495097e-06
34      X36   5.610781  7.885432e-07  1.819169e-03  1.785262e-03  4.134476e-05
65      X67  -4.793322  1.413077e-05  3.259970e-02  3.166707e-02  4.865626e-04
127    X129  -8.412602  2.516574e-09  5.805735e-06  5.757920e-06  2.902868e-07
172    X174  -6.974367  5.603441e-09  1.292714e-05  1.281507e-05  6.155780e-07
...     ...        ...           ...           ...           ...           ...
2044  X2046 -10.842463  1.769295e-14  4.081764e-11  4.071148e-11  5.831091e-12
2048  X2050 -10.983249  4.084836e-15  9.423716e-12  9.415546e-12  3.060886e-12
2113  X2115  -5.069148  7.286580e-06  1.681014e-02  1.640209e-02  2.949148e-04
2144  X2146  -7.567062  7.001831e-10  1.615322e-06  1.604119e-06  9.501897e-08
2245  X2247  -6.050112  3.066047e-07  7.073370e-04  6.956860e-04  1.813685e-05

[72 rows x 6 

En las tablas anteriores se indica cuales son los genes mas significativos osea menores a 0.05 de cada metododologia.


**3.**A continuacion, se realizara una comparación de las medias de las 4 clases con el metodo de pruebas de analisis de varianza ANOVA, utilizando la funcion f_oneway de scipy.stats

In [20]:
from scipy.stats import f_oneway

gen_1 = df[df['y'] == 1].drop(columns='y')
gen_2 = df.drop(columns='y')
gen_3 = df[df['y'] == 3].drop(columns='y')
gen_4 = df.drop(columns='y')

f_stat_an, p_value_an = f_oneway(gen_1, gen_2, gen_3, gen_4)

_, p_bonf, _, _ = multipletests(p_value_an, alpha=0.05, method='bonferroni')
_, p_bh, _, _ = multipletests(p_value_an, alpha=0.05, method='fdr_bh')

genes = df.columns[1:]
resultados_an = pd.DataFrame({
    'Gen': genes,
    'F_stat': f_stat_an,
    'p_value': p_value_an,
    'p_bonf': p_bonf,
    'p_bh': p_bh
})

val_bonf = resultados_an[resultados_an['p_bonf'] < 0.05]
val_bh = resultados_an[resultados_an['p_bh'] < 0.05]

print("Genes significativos utilizando anova y Bonferroni:\n", val_bonf)
print("\nGenes significativos utilizando anova y Benjamini-Hochberg:\n", val_bh)

print('\nGenes significativos con resultados de anova: \n', resultados_an[resultados_an['p_value'] < 0.05])

Genes significativos con Bonferroni:
         Gen     F_stat       p_value    p_bonf          p_bh
0        X2  15.297268  5.834294e-09  0.000013  1.035812e-06
83      X85   8.769972  1.773256e-05  0.040927  4.131361e-04
84      X86  13.451983  5.310526e-08  0.000123  3.714150e-06
106    X108  11.593171  5.148264e-07  0.001188  2.583086e-05
122    X124  16.840008  9.535525e-10  0.000002  3.308794e-07
...     ...        ...           ...       ...           ...
2197  X2199  14.966904  8.634494e-09  0.000020  1.107134e-06
2198  X2200  10.198054  2.920720e-06  0.006741  9.630032e-05
2216  X2218   8.696276  1.947610e-05  0.044951  4.364160e-04
2229  X2231   9.514228  6.904243e-06  0.015935  1.943292e-04
2302  X2304  11.910425  3.482065e-07  0.000804  1.913478e-05

[104 rows x 5 columns]

Genes significativos con Benjamini-Hochberg:
         Gen     F_stat       p_value    p_bonf      p_bh
0        X2  15.297268  5.834294e-09  0.000013  0.000001
8       X10   3.844195  1.055997e-02  1.00000

En estos resultados se puede observar una menor reduccion en la cantidad de variables seleccionadas, exceptuando para los generados exclusivamente con f_oneway.

**4.** Ahora se dividiran los datos en train y test, (50/50), se generaran tres modelos con SVM uno con kernel lineal, uno con kernel polinomial de tercer orden y otro con kernel radial. Seleccionare un determinado numero de variables basado en los mejores resultados anteriores, para asi evitar los tiempos extensos de procesamiento, como esta actividad es de objetivo formativo, se ignorara que se esta cayendo en una fuga de datos.

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

nm = resultados_an[resultados_an['p_value'] < 0.05]['Gen']

x = df[nm]

y = df['y']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

#Modelo kernel lineal
model_lineal = SVC(C= 1, kernel='linear', probability=True)
model_lineal.fit(X_train, y_train)
y_pred_lineal = model_lineal.predict(X_test)

#Modelo kernel polinomial 3er grado
model_polinomial = SVC(C = 1, kernel='poly', degree=3, probability=True)
model_polinomial.fit(X_train, y_train)
y_pred_polinomial = model_polinomial.predict(X_test)

#Modelo kernel radial
model_radial = SVC(C = 1, kernel='rbf', probability=True)
model_radial.fit(X_train, y_train)
y_pred_radial = model_radial.predict(X_test)

print("\nResultados Modelo con Kernel Lineal:")
print(confusion_matrix(y_test, y_pred_lineal))
print(classification_report(y_test, y_pred_lineal))

print("\nResultados Modelo con Kernel Polinomial:")
print(confusion_matrix(y_test, y_pred_polinomial))
print(classification_report(y_test, y_pred_polinomial))

print("\nResultados Modelo con Kernel Radial:")
print(confusion_matrix(y_test, y_pred_radial))
print(classification_report(y_test, y_pred_radial))



Resultados Modelo con Kernel Lineal:
[[ 8  0  0  0]
 [ 0 15  0  0]
 [ 0  0  5  3]
 [ 0  0  0 11]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        15
           3       1.00      0.62      0.77         8
           4       0.79      1.00      0.88        11

    accuracy                           0.93        42
   macro avg       0.95      0.91      0.91        42
weighted avg       0.94      0.93      0.92        42


Resultados Modelo con Kernel Polinomial:
[[ 8  0  0  0]
 [ 0 13  1  1]
 [ 0  0  5  3]
 [ 0  1  1  9]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         8
           2       0.93      0.87      0.90        15
           3       0.71      0.62      0.67         8
           4       0.69      0.82      0.75        11

    accuracy                           0.83        42
   macro avg       0.83      0.83    

Analizando los resultados obtenidos, se puede concluir que el mejor modelo fue el del kernel lineal, ya que obtuvo un f1_score alto en casi todas las clases, y tuvo un accuracy general de 0.93. Antes de llegar a este resultado realice multples particiones de datos para averiguar cual era el mejor valor, ya que al inicio, habia realizado una particion 70/20 y habia observado que me quedaban muy pocas observaciones para el test, por lo tanto, lo cambie a 50/50